In [66]:
from exercise_code.trainer import Trainer
from exercise_code.network import SmoothCrossEntropyLoss
from transformers import PreTrainedTokenizerFast
from transformers import TFAutoModel
from tokenizers import Tokenizer

tokenizer = None


tokenizer = load_pretrained_fast()
# tokenizer = PreTrainedTokenizerFast.from_pretrained('distilbert-base-uncased')
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

hparams = {
    'd_model': 128,
    'd_k': 32,
    'd_v': 32,
    'n_heads': 4,
    'd_ff': 256,
    'n': 3,
    'dropout': 0.1
}


model = Transformer(vocab_size=len(tokenizer),
                    eos_token_id=tokenizer.eos_token_id,
                    hparams=hparams)

In [73]:
optimizer = None
scheduler = None

epochs = 1000
batch_size = 16
import torch.optim as optim

lr_start =1e-4


optimizer = Adam(model.parameters(), lr=lr_start)


loss_func = SmoothCrossEntropyLoss(smoothing=0.1)
file_path = os.path.join(dataset_path, 'dummyDatasets', 'ds_dummy')
collator = CustomCollator(tokenizer=tokenizer)
dataset = CustomIterableDataset(file_path)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collator)

# For Apple M1/M2/M3 users: Try out the MPS framework, it will significantly speed up your training!
if torch.cuda.is_available():
    device = torch.device("cuda")
elif MPS_AVAILABLE:
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
else:
    device = torch.device("cpu")

trainer = Trainer(model=model,
                  loss_func=loss_func,
                  train_loader=dataloader,
                  val_loader=None,
                  optimizer=optimizer,
                  scheduler=scheduler,
                  epochs=epochs,
                  device=device,
                  optimizer_interval=0, 
                  checkpoint_interval=0) 

16


In [74]:
trainer.train()

Training Epoch 709/1000:  25%|██▌       | 1/4 [00:00<00:00, 12.72 batches/s, loss=2.070/2.070, train accuracy=87.680/87.680, learning_rate=1.000e-04]


KeyboardInterrupt: 

In [76]:
def translate(input_sentence, max_iteration_length = 50, probabilistic = False, returns_scores = False):
    # Tokenize input sentence
    encoder_input = torch.tensor(tokenizer.encode(input_sentence))

    # Retrieve output sequence from model
    output_sequence, score_records = model.predict(encoder_input, max_iteration_length, probabilistic, returns_scores)

    # Decode output sequence
    output_sequence = tokenizer.decode(output_sequence, skip_special_tokens=True)

    if returns_scores:
        return output_sequence, score_records
    return output_sequence

In [78]:
output_sequence = translate("Hi, how are you today?", probabilistic=False)
print(output_sequence)

Hallo, wie geht es dir heute?
